In [1]:
import pandas
import numpy as np
from Recommender.DeepContent.Recommender import ContentRecommender


Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
recommender = ContentRecommender('dataset/deep_learning/label_map(boolean_mood).csv', 'dataset/main_song_labels.csv', 'dataset/main_labels.csv', 'dataset/main_user_rating.csv',None,None,None,'Recommender/DeepContent/use5-bestcheckpoint-0.08- 0.35.hdf5','dataset/song_preview/','dataset/deep_learning/label_map(boolean_mood).csv')
result = recommender.analyzeNewSong('/home/capt4ce/projects/major_project/dataset/song_preview/TRAAAAW128F429D538-mzm.jmksdiul.aac.p.m4a')
result.shape

# lModel = SongLabellingModel('Recommender/DeepContent/use5-bestcheckpoint-0.08- 0.35.hdf5','dataset/song_preview/','dataset/deep_learning/label_map(boolean_mood).csv')
# model = lModel.getModel()
# model

True
0
False
a


(1, 50)

In [4]:
result

array([[  5.59105096e-10,   1.66387973e-10,   2.66231287e-10,
          9.65666516e-08,   6.21057858e-11,   2.65521327e-10,
          1.67285251e-07,   1.09928552e-10,   1.60224403e-11,
          8.36470962e-11,   1.17957883e-11,   1.15742194e-10,
          4.89744113e-11,   1.86698157e-11,   4.39599068e-13,
          1.09641470e-10,   7.90774043e-12,   9.38518707e-10,
          4.58757837e-10,   4.39521336e-12,   4.40008663e-12,
          6.72966728e-13,   9.61949357e-12,   2.01099866e-11,
          1.78979661e-11,   6.13958120e-11,   6.83188055e-12,
          5.60954859e-11,   3.50155040e-12,   9.34340019e-12,
          5.50741690e-13,   5.21533305e-09,   8.02132319e-12,
          9.76045460e-12,   1.26516722e-12,   7.98293289e-13,
          1.04216017e-12,   8.81130835e-13,   1.34466593e-09,
          3.66065407e-11,   1.21836976e-11,   3.16821951e-11,
          1.18484172e-11,   1.31788336e-09,   9.96600358e-10,
          1.25853234e-07,   7.24443652e-08,   4.55768543e-08,
        

# Recommendation designing

In [5]:
import pandas
import numpy as np
import math

In [6]:
label_map_path = 'dataset/deep_learning/label_map(boolean_mood).csv'
main_song_label_path = 'dataset/main_song_labels.csv'
main_labels_path = 'dataset/main_labels.csv'
user_rating_path = 'dataset/main_user_rating.csv'

In [8]:
label_map_df = pandas.read_csv(label_map_path, sep='\t')
list(label_map_df.columns)

['track_id',
 'song_id',
 'title',
 'preview_file',
 'Alternative & Punk',
 'Rock',
 'Traditional',
 'Urban',
 'Pop',
 'Other',
 'Western Hip-Hop/Rap',
 'Metal',
 'Western Pop',
 'Electronica',
 'Punk',
 'Indie Rock',
 'Alternative',
 '70s Rock',
 'Adult Alternative Rock',
 'Electric Blues',
 'Country',
 'Jazz',
 'Contemporary R&B/Soul',
 'Alternative Rock',
 'Acoustic Blues',
 '60s Rock',
 'Classic Country',
 'Emo & Hardcore',
 'Mainstream Rock',
 'Classic R&B/Soul',
 'Synth Pop',
 'General Mainstream Rock',
 'Pop Punk',
 'Adult Alternative Pop',
 'Black Metal',
 'Old School Hip-Hop/Rap',
 'Latin Pop',
 'General Latin Pop',
 'Brit Rock',
 'New Wave Pop',
 'Classic Hard Rock',
 'Adult Contemporary',
 'East Coast Rap',
 'European Pop',
 'Latin Rock',
 'Hard Rock',
 'Religious',
 'happiness',
 'sadness',
 'anger',
 'neutral',
 'slow',
 'medium',
 'fast']

## Preparing song label packed

## Preparing Label Vector Lengths

In [ ]:
song_df = pandas.read_csv(main_song_label_path, sep='\t')
song_df.head(5)

In [ ]:
labels_df = pandas.read_csv(main_labels_path, sep='\t')
labels_df['label_count'] = 1
labels_df.head(5)

In [ ]:
TF = labels_df.groupby(['track_id', 'label'], as_index=False).count().rename(columns={'label_count': 'label_count_TF'})
label_distinct = labels_df[['label', 'track_id']].drop_duplicates()
print TF.head(10)
DF = label_distinct.groupby('label', as_index=False).count().rename(columns={'track_id': 'label_count_DF'})
print DF.head(10)
a=math.log10(len(np.unique(labels_df['track_id'])))
DF['IDF']=a-np.log10(DF['label_count_DF'])
print TF.columns
print DF.columns
TF = pandas.merge(TF,DF,on = 'label', how = 'left', sort = False)
TF['TF-IDF']=TF['label_count_TF']*TF['IDF']
TF


In [ ]:
Vect_len = TF[['track_id', 'TF-IDF']]
Vect_len['TF-IDF-Sq'] = Vect_len['TF-IDF']**2
Vect_len = Vect_len.groupby(['track_id'], as_index=False).sum().rename(columns={'TF-IDF-Sq':'TF-IDF-Sq-Sum'})[['track_id', 'TF-IDF-Sq-Sum']]
print Vect_len.columns
Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-Sum']].sum(axis=1))

TF = pandas.merge(TF, Vect_len, on='track_id', how='left', sort=False)
TF['label_wt'] = TF['TF-IDF']/TF['vect_len']
TF.head(3)

## Prepare Users' Profiles

In [ ]:
rating_df = pandas.read_csv(user_rating_path, sep='\t')
rating_df = rating_df[rating_df['rating']!=0]
rating_df

In [ ]:
user_distinct = rating_df['username'].drop_duplicates()
user_profile = pandas.DataFrame()
i=1

for user in user_distinct:
    print(str(i)+' of '+str(len(user_distinct))+' users')
    
    user_data = rating_df[rating_df['username']==user]
    user_data = pandas.merge(user_data,TF, on='track_id', how='inner')
    user_data_processed = user_data.groupby('label', as_index=False).sum().rename(columns={'label_wt': 'label_pref'})[['label', 'label_pref']]
    user_data_processed['user'] = user
    user_profile = user_profile.append(user_data_processed, ignore_index=True)
    i+=1
print len(user_profile)
user_profile.head(5)

## Calculating cosine similarity for each song-user-pair

In [ ]:
distinct_users=np.unique(rating_df['username'])
label_merge_all=pandas.DataFrame()

i=1
for user in distinct_users:
    
    # analizing user data one by one
    user_profile_all= user_profile[user_profile['user']==user]
    distinct_songs = np.unique(TF['track_id'])
    j=1
    for song in distinct_songs:

        if j%300==0:
            print 'song: ', j , 'out of: ', len(distinct_songs) , 'with user: ', i , 'out of: ', len(distinct_users)

        # analizing song one by one
        TF_song= TF[TF['track_id']==song]
        label_merge = pandas.merge(TF_song,user_profile_all,on = 'label', how = 'left', sort = False)
        label_merge['label_pref']=label_merge['label_pref'].fillna(0)
        
        # listing label_value= weight of the label * label profile of the user
        label_merge['label_value']=label_merge['label_wt']*label_merge['label_pref']

        # getting the label weight of the current user-song pair
        label_wt_val=np.sqrt(np.sum(np.square(label_merge['label_wt']), axis=0))
        
        # getting the label value of the current user-song pair
        label_pref_val=np.sqrt(np.sum(np.square(user_profile_all['label_pref']), axis=0))

        # summing the label_value (rating) of user-song pair
        label_merge_final = label_merge.groupby(['user','track_id']).agg({'label_value': 'sum'}).rename(columns = {'label_value': 'score'}).reset_index()

        # score = score / (label weight * label value)
        label_merge_final['score']=label_merge_final['score']/(label_wt_val*label_pref_val)

        label_merge_all = label_merge_all.append(label_merge_final, ignore_index=True)
        j=j+1
    i=i+1
    label_merge_all=label_merge_all.sort_values(by=['user','score']).reset_index(drop=True)
    
label_merge_all.head(10)

In [ ]:
label_merge_all.sort_values(by='score', ascending=False).head(10)

In [ ]:
labels_df[labels_df['track_id']=='TRAQVDI128F42969C8']

In [ ]:
labels_df[labels_df['track_id']=='TRAAAAW128F429D538']

## Recommending (based on user profile)

In [ ]:
rating_df